In [30]:
# Constants
DATA_PATH   = '../data/raw/survey_results_public.csv'
EXPORT_PATH = "../data/processed/1_preprocessed_df.pkl"

REPLACE_DICT = {
    'YearsCodePro': {'Less than 1 year': 0, 'More than 50 years': 51}, 
    'YearsCode':    {'Less than 1 year': 0, 'More than 50 years': 51}, 
    'Age1stCode':   {'Older than 65':86,    'Younger than 5 years':4,
                     '18 - 24 years':21,  '11 - 17 years':14,
                     '5 - 10 years':7, '25 - 34 years': 30,
                     '35 - 44 years':40, '45 - 54 years':50,
                     '55 - 64 years':60,'Older than 64 years':65
                    }}


In [31]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle

___________

## Functions

In [32]:
def split_answers(data_series, delimiter=";"):
    """ 
    Split multiple answers in a single string 
    to a list of single strings each represnting a single answers 

    Parameters:
    * data_series (pd.Series): String series with answers 
    * delimiter (string): Another decimal integer 
                          Defaults to ";"

    Returns: (pd.Series): If column contains 
    """
    
    # Sub functions 
    def is_splittable(pd_series, delimiter):
        """ Check if results multiple should be splitted - Returns boolean """                
        return pd_series.str.contains(delimiter)
    
    def split_answer(pd_series, delimiter): 
        """Function to split single answer"""
        return pd_series.str.split(delimiter)
    
    # --------------------
    
    # Check if multiple answers exist - if none: return original 
    splittable_values = is_splittable(data_series, delimiter)
    if not splittable_values.any():
        return data_series
    
    # Else, split each value to a list 
    modified_series = split_answer(data_series, delimiter)    
    
    # Replace NAs with empty lists 
    mask_null = modified_series.isnull()
    modified_series.loc[mask_null] = modified_series.loc[mask_null].apply(lambda x: [])
    
    return modified_series

___________

# Processing 

## Preprocess Data 

In [33]:
raw_df = pd.read_csv(DATA_PATH)
df = raw_df.copy()

### 1.  Replace Values and parse

In [34]:
for col, replacement in REPLACE_DICT.items():
    df[col] = df[col].replace(replacement).astype(np.float32)

### 2. Split multiple answers 

In [35]:
object_cols = df.select_dtypes(include='object').columns.tolist()
for col in object_cols:
    df[col] = split_answers(df[col])

___________

# Export Data 

In [36]:
df.to_pickle(EXPORT_PATH)

___________